In [3]:
import pygame
import random
import requests
import threading

# --- 1. SETUP ---
WIDTH, HEIGHT = 800, 600
FPS = 60
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "phi3"

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Project 3: LLM Swarm Intelligence")
clock = pygame.time.Clock()
font = pygame.font.SysFont("Arial", 24, bold=True)

# Global State
current_strategy = "SCATTER"  # Default hardcoded strategy
ai_status = "Press SPACE to ask AI for a strategy!"
is_thinking = False

# --- 2. THE AI FUNCTION ---
def ask_hive_mind():
    global current_strategy, ai_status, is_thinking
    is_thinking = True
    ai_status = "Hive Mind is thinking..."
    
    # We ask the AI to pick a strategy based on a scenario
    prompt = """You are the Hive Mind controlling a swarm of 100 drones. 
    A giant predator is attacking the swarm. 
    Choose our survival strategy. You must reply with EXACTLY ONE of these three words:
    1. SCATTER (Run away randomly)
    2. DEFEND (Charge at the predator)
    3. FREEZE (Stop moving to blend in)
    
    Reply with just the word, nothing else."""

    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False
    }

    try:
        response = requests.post(OLLAMA_URL, json=payload)
        result = response.json()["response"].strip().upper()
        
        # Clean up the response just in case the AI talks too much
        if "DEFEND" in result: current_strategy = "DEFEND"
        elif "FREEZE" in result: current_strategy = "FREEZE"
        else: current_strategy = "SCATTER"
        
        ai_status = f"Hive Mind commanded: {current_strategy}!"
    except Exception as e:
        ai_status = "Comm link failed. Defaulting to SCATTER."
        current_strategy = "SCATTER"
        
    is_thinking = False

# --- 3. THE AGENT ---
class Agent:
    def __init__(self):
        self.pos = pygame.math.Vector2(random.randint(0, WIDTH), random.randint(0, HEIGHT))
        self.vel = pygame.math.Vector2(random.uniform(-2, 2), random.uniform(-2, 2))
        self.max_speed = 4

    def update(self, predator_pos):
        dist_to_predator = self.pos.distance_to(predator_pos)
        
        # --- THE LLM DRIVEN LOGIC ---
        if current_strategy == "FREEZE":
            self.vel *= 0.8 # Rapidly slow down to 0
            self.color = (100, 100, 100) # Turn gray to hide
            
        elif current_strategy == "DEFEND":
            self.color = (255, 100, 0) # Turn orange
            if dist_to_predator < 200:
                # Attack vector: Move TOWARDS the predator!
                attack_vector = predator_pos - self.pos
                if attack_vector.length() > 0:
                    attack_vector.scale_to_length(self.max_speed)
                self.vel = self.vel.lerp(attack_vector, 0.05)
                
        else: # SCATTER
            self.color = (0, 255, 150) # Normal green
            if 0 < dist_to_predator < 150:
                # Flee vector: Move AWAY
                flee_vector = self.pos - predator_pos
                flee_vector.scale_to_length(self.max_speed + 2)
                self.vel = self.vel.lerp(flee_vector, 0.1)
            else:
                self.vel += pygame.math.Vector2(random.uniform(-0.5, 0.5), random.uniform(-0.5, 0.5))

        # Enforce Speed Limits & Move
        if self.vel.length() > self.max_speed and current_strategy != "FREEZE":
            self.vel.scale_to_length(self.max_speed)
        self.pos += self.vel

        # Screen Wrap
        if self.pos.x > WIDTH: self.pos.x = 0
        elif self.pos.x < 0: self.pos.x = WIDTH
        if self.pos.y > HEIGHT: self.pos.y = 0
        elif self.pos.y < 0: self.pos.y = HEIGHT

    def draw(self):
        pygame.draw.circle(screen, self.color, (int(self.pos.x), int(self.pos.y)), 5)

# --- 4. MAIN SIMULATION ---
my_swarm = [Agent() for _ in range(100)]
running = True

while running:
    screen.fill((30, 30, 30))
    mouse_pos = pygame.math.Vector2(pygame.mouse.get_pos())

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        # If space is pressed, ask the AI!
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE and not is_thinking:
                threading.Thread(target=ask_hive_mind).start()

    # Draw Predator
    pygame.draw.circle(screen, (255, 50, 50), (int(mouse_pos.x), int(mouse_pos.y)), 15)

    # Update Agents
    for agent in my_swarm:
        agent.update(mouse_pos)
        agent.draw()

    # UI Text
    text = font.render(ai_status, True, (255, 255, 255))
    screen.blit(text, (10, 10))

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()